In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import obspy

from obspy.core.event import Pick, WaveformStreamID, Arrival, Amplitude
from obspy.core.event import Event, Origin, Magnitude
from obspy.core.event import EventDescription, CreationInfo
from obspy import Catalog, UTCDateTime
from obspy import read_events
df=pd.read_pickle('rterg_summary.pkl')

# fixes now implemented in main code
df=df.rename(columns={"Lat.":"Lat", "Long.":"Long","Ehf/Tr^3":"Ehf_Tr3"})
#df["Eventname"] = str(int(df["Eventname"])).zfill(8)   # convert to string with leading zeros
df["iMag"]=pd.to_numeric(df["iMag"], errors='coerce', downcast='float')   # to float (was str)
df

,Eventname,oTime,Lat,Long,Depth,Me,Txo,Ehf,Ebb,Mehf,Ehf_Tr3,Nstats,iMagType,SRC,iMag,TACER_HF,TACER_BB,Comment,mTime,iteration
0,00010500,2000-01-05 07:40:40+00:00,-11.27,165.25,15.0,6.28,83,1.300000e+13,5.900000e+13,6.31,2.300000e+07,19,M_,__,6.0,NaN,NaN,Solomon Islands : IRIS Fetched Data,2016-12-06 22:55:40+00:00,A
1,00010800,2000-01-08 01:19:46+00:00,-9.78,159.99,15.0,6.32,78,1.500000e+13,6.800000e+13,6.34,3.100000e+07,24,M_,__,6.4,NaN,NaN,Solomon Islands : IRIS Fetched Data,2016-12-06 23:28:08+00:00,A
2,00030693,1993-03-06 03:05:48+00:00,-10.73,164.02,15.0,0.00,538,0.000000e+00,-inf,1.00,0.000000e+00,0,M_,__,7.1,NaN,NaN,Solomon Islands : IRIS Fetched Data | 1993,2016-12-07 00:13:59+00:00,A
3,00051699,1999-05-16 00:51:20+00:00,-4.99,152.76,42.0,6.93,90,1.900000e+14,5.600000e+14,7.08,2.600000e+08,27,M_,__,7.0,NaN,NaN,Solomon Islands : IRIS Fetched Data | 1999,2016-12-08 17:45:51+00:00,A
4,00081695,1995-08-16 10:27:26+00:00,-5.51,153.64,45.6,0.00,392,0.000000e+00,-inf,4.00,0.000000e+00,0,M_,__,7.7,NaN,NaN,Solomon Islands : IRIS Fetched Data | 1995,2016-12-08 18:21:34+00:00,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5011,21030702,2021-03-07 19:28:47+00:00,-28.54,-177.01,10.0,5.54,84,2.100000e+12,4.600000e+12,5.78,3.500000e+06,37,M,US,5.5,27,31,NaN,2021-03-07 20:00:42+00:00,6
5012,21030703,2021-03-07 23:56:56+00:00,-29.60,-177.03,10.0,5.88,65,8.600000e+12,1.500000e+13,6.19,3.200000e+07,46,M,US,5.6,34.5,34,NaN,2021-03-08 00:33:27+00:00,6
5013,21030800,2021-03-08 12:56:34+00:00,-4.43,100.98,10.0,5.28,85,1.000000e+12,1.900000e+12,5.56,1.600000e+06,48,M,US,5.5,38.5,64,NaN,2021-03-08 13:27:39+00:00,6
5014,21031000,2021-03-10 20:12:37+00:00,-20.13,-177.67,488.0,6.25,89,3.400000e+13,5.300000e+13,6.58,4.800000e+07,57,M,US,5.8,12,12,NaN,2021-03-10 20:46:15+00:00,6


In [19]:
print(df.Eventname[0])

00010500


In [15]:
# convert dataframe to a dictionary and list all the keys in the dictionary
df2 = df.to_dict()
for key in df2:
    print(key)

Eventname
oTime
Lat
Long
Depth
Me
Txo
Ehf
Ebb
Mehf
Ehf_Tr3
Nstats
iMagType
SRC
iMag
TACER_HF
TACER_BB
Comment
mTime
iteration


In [40]:
def full_test_event(eventname,otime,lat,long,depth,me,txo,ehf,ebb,mehf,Ehf_Tr3,nstats,imagtype,src,imag,tacer_hf,tacer_bb,comment,mtime,
iteration):
    website = 'http://geophysics.eas.gatech.edu/RTerg/'+ str(eventname)
    """
    Function to generate a basic, full test event
    """
    from obspy.core.event import Pick, WaveformStreamID, Arrival, Amplitude
    from obspy.core.event import Event, Origin, Magnitude
    from obspy.core.event import EventDescription, CreationInfo
    from obspy import UTCDateTime
 
    test_event = Event()
    test_event.origins.append(Origin())
    test_event.origins[0].time = UTCDateTime(otime)
    test_event.event_descriptions.append(EventDescription())
    test_event.event_descriptions[0].text = comment
    test_event.origins[0].latitude = lat
    test_event.origins[0].longitude = long
    test_event.origins[0].depth = depth
    test_event.creation_info = CreationInfo(agency_id='NEIC')
    test_event.magnitudes.append(Magnitude())
    test_event.magnitudes[0].mag = me
    test_event.magnitudes[0].magnitude_type = 'Me'
    test_event.magnitudes[0].creation_info = CreationInfo('RTerg')
    test_event.magnitudes[0].origin_id = test_event.origins[0].resource_id
    test_event.magnitudes.append(Magnitude())
    test_event.magnitudes[1].mag = mehf
    test_event.magnitudes[1].magnitude_type = 'Mehf'
    test_event.magnitudes[1].creation_info = CreationInfo('RTerg')
    test_event.magnitudes[1].origin_id = test_event.origins[0].resource_id
    test_event.magnitudes.append(Magnitude())
    test_event.magnitudes[2].mag = imag
    test_event.magnitudes[2].magnitude_type = 'iMag'
    test_event.magnitudes[2].creation_info = CreationInfo('NEIC')
    test_event.magnitudes[2].origin_id = test_event.origins[0].resource_id
 
# Check the magnitude types can be 4 characters long
    # Add ANSS specific attributes
    extra = {'Eventname': {'value': eventname,
                              'type': 'attribute',
                              'namespace': website},
             'Txo': {'value': txo,
                              'type': 'attribute',
                              'namespace': website},
              'Ehf': {'value': ehf,
                              'type': 'attribute',
                              'namespace': website},
              'Ebb': {'value': ebb,
                              'type': 'attribute',
                              'namespace': website},
             'Ehf_Tr3': {'value': Ehf_Tr3,
                              'type': 'attribute',
                              'namespace': website},
             'Nstats': {'value': nstats,
                              'type': 'attribute',
                              'namespace': website},
             'iMagType': {'value': imagtype,
                              'type': 'attribute',
                              'namespace': website},
             'SRC': {'value': src,
                              'type': 'attribute',
                              'namespace': website},
             'TACER_HF': {'value': tacer_hf,
                              'type': 'attribute',
                              'namespace': website},
             'TACER_BB': {'value': tacer_bb,
                              'type': 'attribute',
                              'namespace': website},
             'mTime': {'value': mtime,
                              'type': 'attribute',
                              'namespace': website},
             'iteration': {'value': iteration,
                              'type': 'attribute',
                              'namespace': website}}
    Event.extra = extra # add attributes to event
    catalog = obspy.core.event.Catalog(events=[Event]) # create catalog object

    # Write event if new or updated using obspy write
 #   catalog.write('output_filename', format='QUAKEML',
  #                nsmap={'catalog': 'http://anss.org/xmlns/catalog/0.1'})

 #   print("file was exported to: " + 'output_filename')
    return test_event


In [41]:
#full_test_event(eventname,otime,lat,long,depth,me,txo,ehf,ebb,mehf,Ehf_Tr3,nstats,imagtype,src,imag,tacer_hf,tacer_bb,comment,mtime,iteration)
event= full_test_event(df.Eventname[0],df.oTime[0],df.Lat[0],df.Long[0],df.Depth[0],df.Me[0],df.Txo[0],df.Ehf[0],df.Ebb[0],df.Mehf[0],df.Ehf_Tr3[0],df.Nstats[0],df.iMagType[0],df.SRC[0],df.iMag[0],df.TACER_HF[0],df.TACER_BB[0],df.Comment[0],df.mTime[0],df.iteration[0])
cat = Catalog()
# Link them together.
cat.append(event)
print(cat)
#print(catalog.magnitudes[1])

1 Event(s) in Catalog:
2000-01-05T07:40:40.000000Z | -11.270, +165.250 | 6.28 Me


In [42]:
filename="test_quakeml"
cat.write(filename, format="quakeml")

# QuakeML (xml formatting) 